クロスデバイス　DandMとの併用

In [26]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

5331	東芝ライフスタイル株式会社

In [28]:
today_yymmdd = date.today().strftime("%y%m%d")
print(today_yymmdd)

190527


In [29]:
seed_table = 'segment.d_and_m_segment'
xd_table = 'segment.drawbridge_device'
segment_id = "dm0094" #dm0094_正規雇用ユーザー
uid_table = f"segment.xd_{segment_id}"
print(uid_table)
#イレギュラーな作り方として、既にあるセグメントから一部ユーザーを除外して作成する

segment.xd_dm0094


In [30]:
#一旦、DMセグメントを組み合わせた合成セグメントの中間テーブルを作成する、すぐ消す
tmp_table = "dm_tmp.dm0094_tmp"
print(tmp_table)

dm_tmp.dm0094_tmp


In [34]:
%%slack_notify @yusuke_akada
#XD拡張

#f-string
query = (
    f"""
    select 
    distinct smn_uid
    from segment.d_and_m_segment
    where 
    segment_id= "dm0023"
    and smn_uid not in (select smn_uid from segment.d_and_m_segment where segment_id in ("dm0009","dm0044","dm0069"))
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_tmp = ir.sql_to_pandas(query)
    ir.upload_dataframe(tmp_table, df_tmp, ['bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [35]:
df_tmp.shape[0]

2374002

In [36]:
seed_table = 'segment.d_and_m_segment'
xd_table = 'segment.drawbridge_device'
segment_id = "dm0094" #dm0094_正規雇用ユーザー
uid_table = f"segment.xd_{segment_id}"
print(uid_table)

segment.xd_dm0094


In [37]:
%%slack_notify @yusuke_akada
#XD拡張

#f-string
query = (
    f"""
    select 
    distinct xd2.smn_uid
    from {tmp_table} sd
    left join {xd_table} xd
    on sd.smn_uid = xd.smn_uid
    left join {xd_table} xd2
    on xd.drawbridge_consumer_id = xd2.drawbridge_consumer_id
    where xd2.device in ('iphone'
    ,'android_phone'
    ,'windows_phone'
    ,'windows_computer'
    ,'linux_computer'
    ,'mac'
    ,'ipad'
    ,'android_tablet'
    ,'windows_tablet') 
    and sd.smn_uid <> xd2.smn_uid
    """
    )
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    df_fix = ir.sql_to_pandas(query)
    ir.upload_dataframe(uid_table, df_fix, ['bigint'])

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:実行SQL: 
    s

In [38]:
df_fix.shape[0]

2581197

In [42]:
#過去に作ったものを削除
tables = ["dm_tmp.dm0094_tmp"]
with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
    ir.drop_tables(tables)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: hc3manager02.hc3pro
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.109]
INFO:impyla_service:New connection is opened.
INFO:impyla_service:Dropping tabl